In [42]:
import requests
import json
import lxml
import sys
from datetime import datetime, date

In [6]:
BASE_URL = 'https://oscar.wmo.int/surface/rest/api'
SEARCH_API = BASE_URL + '/search/station?'
STATION_DETAILS = BASE_URL + '/stations/station/'

In [69]:
# we search for Japanese stations
arg = "territoryName=ROU&territoryName=landFixed"

In [70]:
r  = requests.get(SEARCH_API + arg)
if r.status_code != 200:
    print("ERROR: status: {}".format(r.status_code))
    sys.exit(1)

In [71]:
result = r.json()
print("we have {} results".format(len(result)))

we have 167 results


In [72]:
# no modification date
result[0]

{'id': 3656,
 'name': 'ADAMCLISI',
 'region': 'Europe',
 'territory': 'Romania',
 'declaredStatus': 'Operational',
 'latitude': 44.0882700719,
 'longitude': 27.965626028,
 'elevation': 158,
 'stationTypeName': 'Land (fixed)',
 'wigosStationIdentifiers': [{'wigosStationIdentifier': '0-20000-0-15479',
   'primary': True}],
 'stationTypeId': 1,
 'dateEstablished': '1956-07-31T23:00:00.000+0000',
 'stationStatusCode': 'operational',
 'stationTypeCode': 'landOceanSurface',
 'stationProgramsDeclaredStatuses': 'GOS:Operational'}

In [73]:
# loop over result and get the station details for each station
station_details = {}
for station in result:
    internal_id = station["id"]
    r = requests.get(STATION_DETAILS + "{internal_id}/stationReport".format(internal_id=internal_id))
    
    if r.status_code == 200:
        station_details[str(internal_id)] = r.json()
        print("downloaded {internal_id}".format(internal_id=internal_id))
    else:
        print("warning.. could not load station with id {internal_id}".format(internal_id=internal_id))

downloaded 3656
downloaded 3571
downloaded 3567
downloaded 3659
downloaded 3614
downloaded 3564
downloaded 3549
downloaded 3637
downloaded 3511
downloaded 3623
downloaded 3652
downloaded 3553
downloaded 3586
downloaded 3590
downloaded 3569
downloaded 3562
downloaded 3529
downloaded 3542
downloaded 3662
downloaded 3589
downloaded 3526
downloaded 3568
downloaded 3585
downloaded 3530
downloaded 3513
downloaded 3622
downloaded 3608
downloaded 3594
downloaded 3548
downloaded 3641
downloaded 3640
downloaded 3642
downloaded 3620
downloaded 538
downloaded 3658
downloaded 3650
downloaded 3527
downloaded 3552
downloaded 3619
downloaded 3605
downloaded 3653
downloaded 3591
downloaded 3533
downloaded 3647
downloaded 3544
downloaded 3540
downloaded 3657
downloaded 32681
downloaded 3634
downloaded 3520
downloaded 3648
downloaded 3601
downloaded 3618
downloaded 133
downloaded 3507
downloaded 3525
downloaded 3572
downloaded 3630
downloaded 3636
downloaded 3560
downloaded 3545
downloaded 3574
downloade

In [74]:
station_details["3695"]

KeyError: '3695'

In [ ]:
stations_modified = { internal_id: datetime.strptime( station['lastModifiedOn'],"%Y-%M-%d")  for internal_id,station in station_details.items()   }


In [ ]:
my_stations = [ internal_id for internal_id,moddate in stations_modified.items() if moddate > datetime(2019,1,1)   ]

In [ ]:
my_stations

In [ ]:
for idx in my_stations:
    name=station_details[idx]['name']
    print("{name} was modified on {moddate}".format(name=name,moddate=stations_modified[idx]))
    